---

## **START_OF_COLAB_NOTEBOOK_CODE**

In [1]:
!fusermount -u drive
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Colab Notebooks/seoul-bike-transit/route-finder"
%ls -la

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/seoul-bike-transit/route-finder
total 217
-rw------- 1 root root   1666 May 27 06:41 coordinate.py
drwx------ 2 root root   4096 May 27 01:22 data/
-rw------- 1 root root 162371 May 28 02:39 demo.ipynb
-rw------- 1 root root   7257 May 28 01:05 demo.py
-rw------- 1 root root     51 May 19 09:09 .env
-rw------- 1 root root      4 May 27 01:22 .gitignore
-rw------- 1 root root  29603 May 28 03:40 navigator.ipynb
-rw------- 1 root root   5767 May 27 06:41 navigator.py
drwx------ 2 root root   4096 May 28 03:28 __pycache__/
drwx------ 2 root root   4096 May 27 01:22 routes/


In [2]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 163 kB in 4s (39.6 kB/s)
Reading package lists... Don

## **END_OF_COLAB_NOTEBOOK_CODE**

---

In [1]:
colab = False

In [ ]:
%pip install bs4 selenium webdriver_manager python-dotenv haversine pandas tqdm pickle5

In [9]:
from bs4 import BeautifulSoup
from navigator import KakaoRouteFinder
from IPython.display import HTML
import pandas as pd
import navigator
import coordinate
import os

os.environ['WDM_LOG_LEVEL'] = '0'

org, des = ('롯데월드', '뚝섬한강공원')

org_coord = coordinate.get_coordinate(org)
des_coord = coordinate.get_coordinate(des)

(org_x, org_y) = coordinate.transform(org_coord[::-1], 'WGS84', 'WCONGNAMUL')
(des_x, des_y) = coordinate.transform(des_coord[::-1], 'WGS84', 'WCONGNAMUL')

print(f'https://map.kakao.com/?map_type=TYPE_MAP&target=car&rt={int(org_x)},{int(org_y)},{int(des_x)},{int(des_y)}&rt1={org}&rt2={des}')

routes = {}
routes['transit'] = KakaoRouteFinder(colab).find_route_by_keyword('transit', org, des, time_delta=0.5, init_time=3.0)
navigator.extract_route('transit', routes['transit'])

https://map.kakao.com/?map_type=TYPE_MAP&target=car&rt=521675,1114413,515230,1119395&rt1=롯데월드&rt2=뚝섬한강공원



[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 101.0.4951
[WDM] - Get LATEST chromedriver version for 101.0.4951 google-chrome
[WDM] - Driver [/Users/lucetre/.wdm/drivers/chromedriver/mac64_m1/101.0.4951.41/chromedriver] found in cache


[{'time': '31분', 'info': '도보21분 | 환승1회 | 요금 1,250원 | 7.3km'},
 {'time': '35분', 'info': '도보23분 | 환승없음 | 요금 1,200원 | 5.1km'},
 {'time': '41분', 'info': '도보23분 | 환승없음 | 요금 1,200원 | 5.9km'},
 {'time': '37분', 'info': '도보20분 | 환승2회 | 요금 1,250원 | 10.2km'},
 {'time': '37분', 'info': '도보24분 | 환승1회 | 요금 1,200원 | 5.4km'},
 {'time': '42분', 'info': '도보26분 | 환승1회 | 요금 1,250원 | 8.2km'},
 {'time': '46분', 'info': '도보23분 | 환승1회 | 요금 1,250원 | 8.0km'},
 {'time': '39분', 'info': '도보29분 | 환승1회 | 요금 1,250원 | 7.3km'},
 {'time': '40분', 'info': '도보29분 | 환승1회 | 요금 1,250원 | 7.5km'},
 {'time': '57분', 'info': '도보31분 | 환승없음 | 요금 1,200원 | 8.6km'}]

In [10]:
from tqdm import tqdm

soup = BeautifulSoup(routes['transit'], 'html.parser')
S = set()
for route in (soup.find_all('li', {"class": "TransitRouteItem"})):
    nodes = [name.text.strip().replace(" 승차", "").replace(" 하차", "").replace(" 환승", "").replace(" 정류장", "") for name in route.find_all('a', {"data-id": "name"})]
    # print(nodes)
    S.update(nodes)
    nodes = [node.text.strip() for node in route.find_all('li', {"class": "nodeName"})]
    # print(nodes)
    S.update(nodes)

seoul_bike = {}

for lat, lng in [org_coord, des_coord]:
    try:
        _, bike = coordinate.get_nearest_bike(float(lat), float(lng))
        bike_id, bike_name, bike_lat, bike_lng = bike[['대여소\n번호', '보관소(대여소)명', '위도(Y)', '경도(X)']]
        congnamul = coordinate.transform((bike_lng, bike_lat), 'WGS84', 'WCONGNAMUL')
        seoul_bike[bike_id] = ( \
            bike_name, \
            bike_lat, \
            bike_lng, \
            coordinate.get_distance(org_coord, (bike_lat, bike_lng)), \
            coordinate.get_distance(des_coord, (bike_lat, bike_lng)), \
            congnamul,
        )
    except:
        pass

for s in tqdm(S, desc="Retrieve Seoul Bike Stations: "):
    bus = coordinate.df_bus[coordinate.df_bus['정류소명'] == s]
    subway = coordinate.df_subway[coordinate.df_subway['name'] == s.replace('역', '')]
    try:
        if len(subway):
            lat, lng = subway[['lat(y)', 'lng']].iloc[0]
        elif len(bus):
            lat, lng = bus[['Y좌표', 'X좌표']].iloc[0]
        else:
            # print(s, 'N/A')
            continue
        _, bike = coordinate.get_nearest_bike(float(lat), float(lng))
        bike_id, bike_name, bike_lat, bike_lng = bike[['대여소\n번호', '보관소(대여소)명', '위도(Y)', '경도(X)']]
        congnamul = coordinate.transform((bike_lng, bike_lat), 'WGS84', 'WCONGNAMUL')
        seoul_bike[bike_id] = ( \
            bike_name, \
            bike_lat, \
            bike_lng, \
            coordinate.get_distance(org_coord, (bike_lat, bike_lng)), \
            coordinate.get_distance(des_coord, (bike_lat, bike_lng)), \
            congnamul,
        )
    except:
        # print('N/A')
        pass

rows = []
for id in seoul_bike.keys():
    name, lat, lng, org_dist, des_dist, congnamul = seoul_bike[id]
    coord = (lat, lng)
    link = f'https://map.kakao.com/?map_type=TYPE_MAP&target=transit&rt=,,{congnamul[0]},{congnamul[1]}&rt2={name}'
    rows.append([f'<a href="{link}">{name}</a>', coord, org_dist-des_dist, congnamul])

df = pd.DataFrame(rows, columns=["name", "coord", "gap_dist", "congnamul"])
HTML(df.sort_values(by="gap_dist").to_html(escape=False))

Retrieve Seoul Bike Stations: 100%|██████████| 43/43 [00:01<00:00, 30.50it/s]


,name,coord,gap_dist,congnamul
15,롯데월드타워(잠실역2번출구 쪽),"(37.51312637, 127.1009598)",-3.024209,"(522315.0, 1114914.0)"
0,롯데마트 주차장 옆,"(37.51208878, 127.0961914)",-2.887445,"(521261.0, 1114625.0)"
20,잠실역 6번출구,"(37.51408768, 127.0990295)",-2.828819,"(521888.0, 1115181.0)"
26,잠실3거리(갤러리아팰리스),"(37.51050949, 127.0933685)",-2.578394,"(520637.0, 1114187.0)"
21,잠실나루역4번출구뒤,"(37.52080917, 127.1036224)",-2.038955,"(522901.0, 1117047.0)"
5,잠실주공5단지1,"(37.51633453, 127.0938034)",-1.953047,"(520732.0, 1115803.0)"
19,잠실새내역 1번출구,"(37.51129532, 127.0867386)",-1.545941,"(519172.0, 1114403.0)"
24,잠실새내역 5번 출구 뒤,"(37.51198196, 127.0850525)",-1.244008,"(518799.0, 1114593.0)"
13,아시아지하보도 2번 출구,"(37.51216888, 127.0789337)",-0.409409,"(517447.0, 1114644.0)"
16,종합운동장역 4번출구,"(37.51042938, 127.071373)",0.247144,"(515776.0, 1114160.0)"


In [11]:
finder = KakaoRouteFinder(colab)
bikes = sorted(seoul_bike.items(), key=lambda item: item[1][3]-item[1][4])
bike_routes = {}
for i, srt in tqdm(enumerate(bikes), desc="Finding Bike Routes: "):
    (_, (srt_name, srt_lat, srt_lng, _, _, srt_coord)) = srt
    for j in tqdm(range(i+1, len(bikes))):
        end = bikes[j]
        (_, (end_name, end_lat, end_lng, _, _, end_coord)) = end
        route = finder.find_route_by_congnamul('bike', srt_coord, end_coord, rt1=str(i), rt2=str(j), verbose=False, init=False, time_delta=0.2)
        route_id = f'{i}-{j}'
        bike_routes[route_id] = navigator.extract_route('bike', route)
del finder

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 101.0.4951
[WDM] - Get LATEST chromedriver version for 101.0.4951 google-chrome
[WDM] - Driver [/Users/lucetre/.wdm/drivers/chromedriver/mac64_m1/101.0.4951.41/chromedriver] found in cache
100%|██████████| 1/1 [00:00<00:00,  2.27it/s]]
0it [00:00, ?it/s]s: : 29it [03:22,  1.53s/it]
Finding Bike Routes: : 30it [03:22,  6.74s/it]


In [12]:
finder = KakaoRouteFinder(colab)

org_routes = []
des_routes = []

for i, bike in tqdm(enumerate(bikes), desc="Finding Transit Routes"):
    (_, (name, _, _, _, _, coord)) = bike
    try:
        route = finder.find_route_by_congnamul('transit', (org_x, org_y), coord, rt1=org, rt2=str(i)+name, verbose=False, init=True, time_delta=0.5, init_time=2.2)
        org_routes.append(navigator.extract_route('transit', route))
    except:
        route = finder.find_route_by_congnamul('walk', (org_x, org_y), coord, rt1=org, rt2=str(i)+name, verbose=False, init=False, time_delta=0.5)
        org_routes.append(navigator.extract_route('walk', route))
        
    try:
        route = finder.find_route_by_congnamul('transit', coord, (des_x, des_y), rt1=str(i)+name, rt2=des, verbose=False, init=True, time_delta=0.5, init_time=2.2)
        des_routes.append(navigator.extract_route('transit', route))
    except:
        route = finder.find_route_by_congnamul('walk', coord, (des_x, des_y), rt1=str(i)+name, rt2=des, verbose=False, init=False, time_delta=0.5)
        des_routes.append(navigator.extract_route('walk', route))

del finder

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 101.0.4951
[WDM] - Get LATEST chromedriver version for 101.0.4951 google-chrome
[WDM] - Driver [/Users/lucetre/.wdm/drivers/chromedriver/mac64_m1/101.0.4951.41/chromedriver] found in cache
Finding Transit Routes: 30it [03:03,  6.11s/it]


In [13]:
import pickle5 as pickle
filename = f'routes/{org}-{des}.pkl'

with open(filename, 'wb') as f:
    data = {'org_routes': org_routes, "des_routes": des_routes, "bikes": bikes, "bike_routes": bike_routes }
    pickle.dump(data, f)